In [ ]:
library(data.table)

In [ ]:
setwd(project_directory) # set wd to project directory containing all the sub folder

### Location enrichment analysis (genes or cells)

In [ ]:
test_loc_enrichment=function(data,lk_thres=NULL,p_thres=NULL,sub_col){
    if (!is.null(lk_thres)){dat_filt=data[lk_norm>=lk_thres]}else{dat_filt=data}
    if (!is.null(p_thres)){dat_filt=dat_filt[emp_pval_adjust_BH<=p_thres]}
    
    cts=unique(dat_filt$ClusterName)
    anats=unique(dat_filt$layer)
    
    ftest_res=data.table()
    for (ict in cts){
        for (ianat in anats){
           # print(paste0(ict," ",ianat))
            cont_mat=matrix(c(dat_filt[ClusterName==ict&layer==ianat,length(unique(get(sub_col)))], 
                              dat_filt[ClusterName!=ict&layer==ianat,length(unique(get(sub_col)))], 
                              dat_filt[ClusterName==ict&layer!=ianat,length(unique(get(sub_col)))], 
                              dat_filt[ClusterName!=ict&layer!=ianat,length(unique(get(sub_col)))]),
                            nrow = 2,
                            dimnames = list(anat = c("yes", "no"),
                                            ct = c("yes", "no")))
            ft=fisher.test(cont_mat,alternative = "greater",conf.int = TRUE)
            ftest_res=rbindlist(list(ftest_res,data.table(cell_type=ict,
                                                          layer=ianat,
                                                          p.value=ft$p.value,
                                                          cof.int.low=ft$conf.int[1],
                                                          cof.int.high=ft$conf.int[2])))
        }
    }
    return(ftest_res)
}

### MOB cell type enrichment analysis

In [ ]:
anat=fread("MOB/CN13_D2_barcodes_under_tissue_annot.tsv")
setnames(anat,"poly.ID","layer")

In [ ]:
thres=c(0.01,0.1) #p-value, lk_norm

#### 1x (hd)

In [ ]:
lk=fread("results/ct_likelihoods_li_hd_fastp_1000.tsv")
tag="li_hd"

In [ ]:
lk=fread("results/ct_likelihoods_unmodgtf_li_hd_fastp_1000.tsv")
tag="unmodgtf_li_hd"

In [ ]:
#prepare
lk[,N_ct:=sum(lk_rat>=0.8&lk_norm>=thres[2]&emp_pval_adjust_BH<=thres[1]),by=c("x","y")]
lk_anat=merge(lk[lk_rat==1&N_ct==1],anat,by.x=c("x","y"),by.y=c("spot_x","spot_y"))

#### binned 

In [ ]:
lk=fread("results/ct_likelihoods_li_low_fastp_1000.tsv")
coords=fread("MOB_binned/hdst-lowres-coordinates.csv")
tag="li_low"

In [ ]:
lk=fread("results/ct_likelihoods_unmodgtf_li_low_fastp_1000.tsv")
coords=fread("MOB_binned/hdst-lowres-coordinates.csv")
tag="unmodgtf_li_low"

In [ ]:
#prepare
sub=lk[bin=="5x"]
sub[,N_ct:=sum(lk_rat>=0.8&lk_norm>=thres[2]&emp_pval_adjust_BH<=thres[1]),by=c("x","y")]
lk_anat=merge(sub[lk_rat==1&N_ct==1],
                       unique(coords[,c("region","5x_x","5x_y"),with=FALSE][!duplicated(cbind(`5x_x`,`5x_y`))]),
                       by.x=c("x","y"),
                       by.y=c("5x_x","5x_y"))
setnames(lk_anat,"region","layer")

#### segmentd

In [ ]:
lk=fread("results/ct_likelihoods_li_seg_fastp_1000.tsv")
tag="li_seg"

In [ ]:
lk=fread("results/ct_likelihoods_unmodgtf_li_seg_fastp_1000.tsv")
tag="unmodgtf_li_seg"

In [ ]:
#prepare
lk[,N_ct:=sum(lk_rat>=0.8&lk_norm>=thres[2]&emp_pval_adjust_BH<=thres[1]),by=c("x","y")]
lk_anat=merge(lk[lk_rat==1&N_ct==1],anat,by.x=c("x","y"),by.y=c("spot_x","spot_y"))

#### here actually run

In [ ]:
#enrichment analyis
res_li=test_loc_enrichment(data = lk_anat,p_thres = thres[1],lk_thres = thres[2],sub_col = "bc")
res_li[,p.value.adjust:=p.adjust(p.value,method = "BY"),]
write.table(res_li,paste0("results/ct_anat_enrich_",tag,".tsv"),sep="\t", quote=FALSE,row.names=FALSE)

### Breast cancer cell type enrichment analysis

In [ ]:
anat=fread("BC/CN21_BC24350_E2_barcodes_under_tissue_annot.tsv")
setnames(anat,"poly.ID","layer")

In [ ]:
thres=c(0.05,0.7) #p-value, lk_norm

#### 1x (hd)

In [ ]:
lk=fread("results/ct_likelihoods_E2_tnbc_hd.tsv")
tag="E2_tnbc_hd"

In [ ]:
lk=fread("results/ct_likelihoods_E2_unmodgtf_tnbc_hd.tsv")
tag="E2_unmodgtf_tnbc_hd"

In [ ]:
#prepare
lk[,N_ct:=sum(lk_rat>=0.8&lk_norm>=thres[2]&emp_pval_adjust_BH<=thres[1]),by=c("x","y")]
lk_anat=merge(lk[lk_rat==1&N_ct==1],anat,by.x=c("x","y"),by.y=c("spot_x","spot_y"))

#### binned 

In [ ]:
lk=fread("results/ct_likelihoods_E2_tnbc_low.tsv")
coords=fread("BC_binned/hdst-breast-cancer-lowres-coordinates.csv")
tag="E2_tnbc_low"

In [ ]:
lk=fread("results/ct_likelihoods_E2_unmodgtf_tnbc_low.tsv")
coords=fread("BC_binned/hdst-breast-cancer-lowres-coordinates.csv")
tag="E2_unmodgtf_tnbc_low"

In [ ]:
#prepare
sub=lk[bin=="5x"]
sub[,N_ct:=sum(lk_rat>=0.8&lk_norm>=thres[2]&emp_pval_adjust_BH<=thres[1]),by=c("x","y")]
lk_anat=merge(sub[lk_rat==1&N_ct==1],
                       unique(coords[,c("region","5x_x","5x_y"),with=FALSE][!duplicated(cbind(`5x_x`,`5x_y`))]),
                       by.x=c("x","y"),
                       by.y=c("5x_x","5x_y"))
setnames(lk_anat,"region","layer")

#### segmentd

In [ ]:
lk=fread("results/ct_likelihoods_E2_tnbc_seg.tsv")
tag="E2_tnbc_seg"

In [ ]:
lk=fread("results/ct_likelihoods_E2_unmodgtf_tnbc_seg.tsv")
tag="E2_unmodgtf_tnbc_seg"

In [ ]:
#prepare
lk[,N_ct:=sum(lk_rat>=0.8&lk_norm>=thres[2]&emp_pval_adjust_BH<=thres[1]),by=c("x","y")]
lk_anat=merge(lk[lk_rat==1&N_ct==1],anat,by.x=c("x","y"),by.y=c("spot_x","spot_y"))

#### here actually run

In [ ]:
#enrichment analyis
res_li=test_loc_enrichment(data = lk_anat,p_thres = thres[1],lk_thres = thres[2],sub_col = "bc")
res_li[,p.value.adjust:=p.adjust(p.value,method = "BY"),]
write.table(res_li,paste0("results/ct_anat_enrich_",tag,".tsv"),sep="\t", quote=FALSE,row.names=FALSE)

### MOB enrichment of HDST layer specific genes in layers specific genes according to ABA

In [ ]:
#aba genes
aba_layer_spec=fread("ext_data/aba_MOB_diff.tsv")

In [ ]:
#hdst genes
hdst_markers=fread("DE/Table_DEGs.csv",drop = "V1")
setnames(hdst_markers,names(hdst_markers),c("gene","lfc","p.value","ClusterName"))
pthres=0.1
tag=""

In [ ]:
#hdst genes
hdst_markers=fread("DE/mob_region_de_unmodgtf.csv")
setnames(hdst_markers,names(hdst_markers),c("gene","lfc","p.value","ClusterName"))
pthres=0.05
tag="_unmodgtf"

In [ ]:
hdst_aba=merge(hdst_markers[lfc>1.5& p.value<pthres],aba_layer_spec[`fold-change`>1.5],by="gene",all=FALSE)

In [ ]:
#remove ONL because it doesn't exist in ABA
hdst_aba=hdst_aba[!ClusterName=="Olfactory Nerve Layer (ONL)"]
#combine some layers to match ABA annotation
hdst_aba[ClusterName%in%c("Granule Cell Layer External (GCL-E)","Granule Cell Layer Internal (GCL-I)","Rostral Migratory System (RMS)","Ependymal Cell Zone (E)"),ClusterName:="Granule Layer (GR)",]

In [ ]:
hdst_aba_enrich=test_loc_enrichment(hdst_aba,sub_col = "gene")
hdst_aba_enrich[,p.adjust:=p.adjust(p.value,method = "BY"),]

In [ ]:
write.table(hdst_aba_enrich,paste0("results/aba_deg_enrich",tag,".tsv"),sep="\t", quote=FALSE,row.names=FALSE)